In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import bigrams,trigrams
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import re
from heapq import nlargest

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('all-corpora')  # use in case you are unsure what to download so download all

[nltk_data] Downloading package wordnet to /home/deshwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/deshwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/deshwal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stopword = set(stopwords.words('english'))
stopword.update(['amp','cc','rt','@handle']) # rt = retweet

stemmer = SnowballStemmer('english')
lemma = WordNetLemmatizer()

# Input

In [4]:
df = pd.read_csv('train_tweets.txt', sep="\t", header=None,names=['id','tweet'])

In [5]:
df.head()

,id,tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


# Helpers

In [6]:
def remove_stopwords(text):
    return ' '.join([word.lower() for word in text.split(' ') if word.lower() not in stopword and len(word)>2])

In [7]:
def get_vocab(data_frame,col_name):
    vocab = {}
    for tweet in tqdm(data_frame[col_name]):
        for word in tweet.split(' '):
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [28]:
def text_cleaner(text):
    
    # text = re.sub(r'^https?:\/\/.*[\r\n]*', '-URL-', text, flags=re.MULTILINE) # sub URL
    text = re.sub(r'http\S+', ' ', text) 
    text = re.sub(r'www\S+', ' ', text) 
    
    text = re.sub(r'\d',' ',text) # remove any number
    
    # text = re.sub('@handle',' ',text)
    text = re.sub(r'@\w+', '', text)
    
    text = re.sub('-',' ',text)
    text = re.sub('_',' ',text)
    
    # text = re.sub(r'([\W_])\1+',' ',text) # multiple repeating special characters
    text = re.sub(r'(?<=\s)[\W\d](?=(\s|$))', ' ', text)
    text = re.sub(r'(?<=\w)\W+(?=(\s|$))', ' ', text)
    text = re.sub(r'(\W)\1+(?=\w)', r'\1', text)
    
    text = re.sub('\n',' ',text)
    text = re.sub('\s\s+',' ',text) # multiple space
    text = text.lstrip()
    text = text.rstrip()
    return text   

In [9]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", 
"could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", 
"hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", 
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", 
"I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
"i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", 
"it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", 
"let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
"mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
"needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", 
"oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
"she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
 "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
 "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
 "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
 "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
 "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
 "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
 "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  
 "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
 "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
 "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
 "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
 "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
 "y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
 "you'll've": "you will have", "you're": "you are", "you've": "you have", "ya'll": "you all" ,"let's":'let us'}

def remove_contraction(text):
    for word in text.split(' '):
        if word.lower() in contraction_dict:
            text = text.replace(word, contraction_dict[word.lower()])  
    return text

# Pre Processing

In [10]:
df['tweet'] = df['tweet'].apply(lambda text: remove_contraction(text)) # remove contractions
df['tweet'] = df['tweet'].apply(lambda text: remove_stopwords(text)) # remove stopwords
df['tweet'] = df['tweet'].apply(lambda text: text_cleaner(text)) #  clean text

df['tweet'] = df['tweet'].apply(lambda text: ' '.join([lemma.lemmatize(word) for word in text.split(' ')]))
# lemmatize

In [11]:
vocab = get_vocab(df,'tweet')

100%|██████████| 328195/328195 [00:01<00:00, 291491.80it/s]


In [19]:
sorted_vocab = nlargest(len(vocab),vocab,vocab.get)

In [37]:
sorted_vocab[-443]

'suggestion-everything'

In [35]:
vocab['technic']

1